# Basic Recommendation System

By Justin Wong

## Poll Results

![Poll](screenshots/poll.png "poll")


# What We'll Do

1. Go over simple recommendation systems for movies.

2. Blockchain Application to execute transaction and send funds from one account to another, using local blockcahin(Ganache)


In [176]:
import numpy as np
import pandas as pd

#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Parse the stringified features into their corresponding python objects
from ast import literal_eval

import pickle

###  Recommendation Systems

Recommendation systems are good and most useful when you have a lot of data. We'll be using a movies dataset as a baseline to find movies that are similar to each other.

We'll be building:

- `Simple recommender`: offer generalized recommendations to every user, based on movie popularity and/or genre. The basic idea behind this system is that movies that are more popular and critically acclaimed will have a higher probability of being liked by the average audience. 

- `Content-based recommender`: suggest similar items based on a particular item. The general idea behind these recommender systems is that if a person likes a particular item, he or she will also like an item that is similar to it. 

In [177]:
#  downloaded  from https://www.kaggle.com/rounakbanik/the-movies-dataset
!ls data

credits.csv         links.csv           movies_metadata.csv ratings_small.csv
keywords.csv        links_small.csv     ratings.csv


In [121]:
fn = 'data/movies_metadata.csv'
movies_metadata = pd.read_csv(fn, low_memory=False)
movies_metadata.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [175]:
garbo = [print(col) for col in movies_metadata.columns]

index
adult
belongs_to_collection
budget
genres
homepage
id
imdb_id
original_language
original_title
overview
popularity
poster_path
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
video
vote_average
vote_count
cast
crew
keywords
director
soup


## Simple Recommender Based on Rankings/Reports

### Scoring our Rankings

$$ WeightedRating = \left(\frac{v}{v+m}*R\right) + \left(\frac{m}{v+m}*C\right)$$

v = number of votes for movie

m = minimum votes required to be listed in chart

R is average rating of movie

C is mean vote across whole report

In [122]:
# Calculate mean of vote average column
C = movies_metadata['vote_average'].mean()

# Calculate the minimum number of votes required to be in the chart, m
m = movies_metadata['vote_count'].quantile(0.90)

print("Vote average: {C}".format(C=C))
print("Vote count 90percential: {m}".format(m=m))


Vote average: 5.618207215133889
Vote count 90percential: 160.0


In [123]:
# Filter out all qualified movies into a new DataFrame
q_movies = movies_metadata.copy().loc[movies_metadata['vote_count'] >= m]

print("filtered for top  90% shape: {s}".format(s=q_movies.shape))
print("Full dataset shape: {s}".format(s=movies_metadata.shape))

filtered for top  90% shape: (4555, 24)
Full dataset shape: (45466, 24)


In [124]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)


In [178]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)


In [179]:
#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


## Comparing with IMDB

Top  movies: https://www.imdb.com/chart/top/

![Top 10 from IMDB](screenshots/imdb_top_10.png "Top 10 from IMDB")

## Content-Based Recommender

Recommendations are based on movies that are similar to a particular movie.


In [181]:
#Print plot overviews of the first 5 movies.
movies_metadata['overview'].sample(5)

20830    During the 1920s, many impoverished Jews searc...
34220    Varian Fry rescues more than 2,000 artists fro...
31828    In Las Vegas for a quicky divorce, a just-paro...
30383    The uncle of an executed murderess relates fou...
28434    A romantic drama about a woman who enters into...
Name: overview, dtype: object

In [191]:
s = movies_metadata[movies_metadata['title']=='Transformers']['overview']
print(s)

12034    Young teenager, Sam Witwicky becomes involved ...
Name: overview, dtype: object


## NLP

We have an NLP problem  here, where  we need to extract some features from the overviews to accurately categorize the different movies into appropriate groups based on similarity/dissimilarity.

We'll need to compute word vectors for each overview/document. To do so, we'll compute



In [128]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
movies_metadata['overview'] = movies_metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies_metadata['overview'])


In [129]:
print("Shape of TFIDF matrix: {s}".format(s=tfidf_matrix.shape))

Shape of TFIDF matrix: (45466, 75827)


From the above output, you observe that in the 45,000 movies, there is a 75,827 size vocabularies or words in  our dataset.

In [195]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[74000:74010]

['yatsufusa',
 'yatterman',
 'yau',
 'yauch',
 'yavakri',
 'yavin',
 'yaya',
 'yayoi',
 'yayá',
 'yazgı']

We can compute similarity scores using different mathematical funtions, such as manhattan,  euclidean, cosine, etc.

![Cosine Similarity](screenshots/cos_similarity.png "Cosine similarity")

In [131]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [196]:
print("Shape of cosine_sim: {s}".format(s=cosine_sim.shape))

print(cosine_sim[1]) ## to see what it actually looks like

Shape of cosine_sim: (45466, 45466)
[0.01504121 1.         0.04681953 ... 0.         0.02198641 0.00929411]


## Define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies.

In [197]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(movies_metadata.index, index=movies_metadata['title']).drop_duplicates()

In [198]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [201]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_metadata['title'].iloc[movie_indices]

In [203]:
display(get_recommendations('Transformers'))
# display(get_recommendations('The Godfather'))

15761              The Kautokeino Rebellion
26627    Vi hade i alla fall tur med vädret
160                               Desperado
25872      The Mountain of the Cannibal God
27020            See Here, Private Hargrove
3429                                 Taffin
22770       Το Ξύλο Βγήκε Από Τον Παράδεισο
43473                           Jump London
31014                                 Storm
30554                            The Garden
Name: title, dtype: object

## How we can do better?

Adding additional "metadata", such as

- credits
- genres
- keyords

In [226]:
!ls data

credits.csv         links.csv           movies_metadata.csv ratings_small.csv
keywords.csv        links_small.csv     ratings.csv


In [227]:
movies_metadata.iloc[[19730, 29503, 35587]]

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,soup,cast_y,crew_y,keywords_y,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y
19730,11666,False,NaN,20000000,"[action, adventure, comedy]",NaN,9760,tt0799949,en,Epic Movie,...,kalpenn adamcampbell jennifercoolidge jasonfr...,"[{'cast_id': 1, 'character': 'Edward', 'credit...","[{'credit_id': '56348a269251412857016e9e', 'de...",[],"[Kal Penn, Adam Campbell, Jennifer Coolidge]","[{'credit_id': '56348a269251412857016e9e', 'de...",[],"[{'cast_id': 1, 'character': 'Edward', 'credit...","[{'credit_id': '56348a269251412857016e9e', 'de...",[]
29503,13387,False,NaN,0,"[thriller, mystery]",NaN,141971,tt1180333,fi,Blackout,...,petterisummanen ismokallio eppusalminen jpsii...,"[{'cast_id': 1, 'character': 'Pekka', 'credit_...","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",[],"[Petteri Summanen, Ismo Kallio, Eppu Salminen]","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",[],"[{'cast_id': 1, 'character': 'Pekka', 'credit_...","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",[]
35587,13395,False,NaN,0,"[thriller, mystery]",NaN,141971,tt1180333,fi,Blackout,...,petterisummanen ismokallio eppusalminen jpsii...,"[{'cast_id': 1, 'character': 'Pekka', 'credit_...","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",[],"[Petteri Summanen, Ismo Kallio, Eppu Salminen]","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",[],"[{'cast_id': 1, 'character': 'Pekka', 'credit_...","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",[]


In [228]:
# Load keywords and credits
credits_fn='data/credits.csv'
keywords_fc='data/keywords.csv'
credits = pd.read_csv(credits_fn)
keywords = pd.read_csv(keywords_fc)

# Remove rows with bad IDs.
movies_metadata = movies_metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies_metadata['id'] = movies_metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
movies_metadata = movies_metadata.merge(credits, on='id')
movies_metadata = movies_metadata.merge(keywords, on='id')

In [229]:
# Print the first two movies of your newly merged metadata
movies_metadata.head(2)

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,keywords_y,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y,cast,crew,keywords
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,...,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


### Data Cleaning

Data is present in the form of "stringified" lists. You need to convert them into a way that is usable for you.

1. Extract out certain fields like director,  first 3 cast members, keywords, and genres.
2. Making everything lower case.
3. Removing unnecessary spaces.

In [230]:
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    movies_metadata[feature] = movies_metadata[feature].apply(literal_eval)

ValueError: malformed node or string: ['animation', 'comedy', 'family']

In [ ]:
# Get the director's name from the crew feature
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Return the top 3 elements or the entire list
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
movies_metadata['director'] = movies_metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    movies_metadata[feature] = movies_metadata[feature].apply(get_list)

In [ ]:
# Print the new features of the first 3 films
movies_metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    movies_metadata[feature] = movies_metadata[feature].apply(clean_data)

### Combining everything together

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
# Create a new soup feature
movies_metadata['soup'] = movies_metadata.apply(create_soup, axis=1)

In [149]:
movies_metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


In [150]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movies_metadata['soup'])

In [151]:
print("Shape of Count Vectorizer matrix: {s}".format(s=count_matrix.shape))

Shape of Count Vectorizer matrix: (46628, 73881)


In [152]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)


In [153]:
# Reset index of your main DataFrame and construct reverse mapping as before
movies_metadata = movies_metadata.reset_index()
indices = pd.Series(movies_metadata.index, index=movies_metadata['title'])

In [154]:
display(get_recommendations('The Dark Knight Rises', cosine_sim2))
display(get_recommendations('The Dark Knight Rises'))

display(get_recommendations('The Godfather', cosine_sim2))
display(get_recommendations('The Godfather'))

12589      The Dark Knight
10210        Batman Begins
9311                Shiner
9874       Amongst Friends
7772              Mitchell
516      Romeo Is Bleeding
11463         The Prestige
24090            Quicksand
25038             Deadfall
41063                 Sara
Name: title, dtype: object

3777                 A Couch in New York
40653                              Wacko
38251                             Agyaat
1304                    April Fool's Day
16844                 A Hole in the Soul
43127                     Hunting Season
16510                   Morsian yllättää
19970          H.P. Lovecraft's The Tomb
10230    Me and You and Everyone We Know
16042                    The Last Letter
Name: title, dtype: object

1934            The Godfather: Part III
1199             The Godfather: Part II
15609                   The Rain People
18940                         Last Exit
34488                              Rege
35802            Manuscripts Don't Burn
35803            Manuscripts Don't Burn
8001     The Night of the Following Day
18261                 The Son of No One
28683            In the Name of the Law
Name: title, dtype: object

25374                              The Fearmakers
3844                   Sorority House Massacre II
3244                                    Jail Bait
20667                         The Goddess of 1967
33253                                 Son of Saul
37854                                          RR
39265    Texas - Doc Snyder hält die Welt in Atem
37097                                  Reptilicus
23349          Celestial Wives of the Meadow Mari
183                                   Nine Months
Name: title, dtype: object

In [ ]:
pickle.dump(count_matrix, open('cos_CountVectorizer.pickle','wb'))

# References:

- https://www.datacamp.com/community/tutorials/recommender-systems-python

- https://medium.com/@MAbdElRaouf/simple-content-based-recommendation-engine-flask-api-heroku-dd27760dfe8e

## IN  PROGRESS

In [170]:
def recommended_shows(title):
    
    #Get show index
    title_iloc = movies_metadata.index[movies_metadata['title'] == title][0]
    
    #Get cosine similarity
    show_cos_sim = cosine_similarity(movies_metadata[title_iloc], movies_metadata).flatten()
    
    #Get the top 5 most similar shows
    sim_titles_vects = sorted(list(enumerate(show_cos_sim)), key=lambda x: x[1], reverse=True)[1:6]
    
    #Return result
    response = '\n'.join([f'{movies_metadata.iloc[t_vect[0]][0]} --> confidence: {round(t_vect[1],1)}' for t_vect in sim_titles_vects])
    
    return response
# print(recommended_shows('The Godfather'))

def get_recommendations_response(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_metadata.iloc[movie_indices]

In [103]:
r = get_recommendations('The Godfather')

In [166]:
garbo = [print(x) for  x  in r]

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_metadata['title'].iloc[movie_indices]

The Godfather: Part II
The Godfather Trilogy: 1972-1990
The Godfather: Part III
Blood Ties
Household Saints
Start Liquidation
Election
A Mother Should Be Loved
Short Sharp Shock
Beck 28 - Familjen


In [171]:
rz = get_recommendations_response('The Godfather')

In [172]:
rz

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director,soup
1934,1934,False,"{'id': 230, 'name': 'The Godfather Collection'...",54000000,"[crime, drama, thriller]",http://www.imdb.com/title/tt0099674/,242,tt0099674,en,The Godfather: Part III,...,All the power on earth can't change destiny.,The Godfather: Part III,False,7.1,1589.0,"[alpacino, dianekeaton, andygarcía]","[{'credit_id': '52fe422bc3a36847f8009737', 'de...","[italy, christianity, newyork]",francisfordcoppola,italy christianity newyork alpacino dianekeato...
1199,1199,False,"{'id': 230, 'name': 'The Godfather Collection'...",13000000,"[drama, crime]",NaN,240,tt0071562,en,The Godfather: Part II,...,"I don't feel I have to wipe everybody out, Tom...",The Godfather: Part II,False,8.3,3418.0,"[alpacino, robertduvall, dianekeaton]","[{'credit_id': '52fe422bc3a36847f80094dd', 'de...","[italo-american, cuba, vororte]",francisfordcoppola,italo-american cuba vororte alpacino robertduv...
15609,15609,False,NaN,0,[drama],NaN,59231,tt0064873,en,The Rain People,...,NaN,The Rain People,False,5.5,10.0,"[jamescaan, shirleyknight, robertduvall]","[{'credit_id': '52fe4986c3a36847f819ee5b', 'de...",[roadmovie],francisfordcoppola,roadmovie jamescaan shirleyknight robertduvall...
18940,18940,False,NaN,0,"[crime, drama, thriller]",NaN,273807,tt0375909,en,Last Exit,...,NaN,Last Exit,False,0.0,0.0,[],[],[],,crime drama thriller
34488,34488,False,NaN,0,"[thriller, crime, drama]",NaN,313653,tt4019578,en,Rege,...,NaN,Rege,False,8.0,1.0,[],[],[],,thriller crime drama
35802,35802,False,NaN,0,"[crime, drama]",NaN,191731,tt2912144,fa,دست‌نویس‌ها نمی‌سوزند,...,NaN,Manuscripts Don't Burn,False,6.1,8.0,[],"[{'credit_id': '52fe4c909251416c910f8ea5', 'de...",[],mohammadrasoulof,mohammadrasoulof crime drama
35803,35803,False,NaN,0,"[crime, drama]",NaN,191731,tt2912144,fa,دست‌نویس‌ها نمی‌سوزند,...,NaN,Manuscripts Don't Burn,False,6.1,8.0,[],"[{'credit_id': '52fe4c909251416c910f8ea5', 'de...",[],mohammadrasoulof,mohammadrasoulof crime drama
8001,8001,False,NaN,0,"[crime, drama]",NaN,46495,tt0064728,en,The Night of the Following Day,...,"The Higher the Stakes, The Greater the Terror.",The Night of the Following Day,False,5.3,10.0,"[marlonbrando, richardboone, ritamoreno]","[{'credit_id': '52fe46fac3a36847f811d1ab', 'de...",[kidnapping],hubertcornfield,kidnapping marlonbrando richardboone ritamoren...
18261,18261,False,NaN,15000000,"[drama, thriller, crime]",NaN,74536,tt1535612,en,The Son of No One,...,Serve. Protect. Lie.,The Son of No One,False,4.8,93.0,"[channingtatum, alpacino, juliettebinoche]","[{'credit_id': '52fe48d0c3a368484e10c49f', 'de...",[],ditomontiel,channingtatum alpacino juliettebinoche ditomo...
28683,28683,False,NaN,0,"[crime, drama]",NaN,195382,tt0041506,it,In nome della legge,...,NaN,In the Name of the Law,False,7.0,2.0,"[massimogirotti, jonesalinas, camillomastrocin...","[{'credit_id': '52fe4d0d9251416c91109087', 'de...",[italy],pietrogermi,italy massimogirotti jonesalinas camillomastro...


In [173]:
'\n'.join([{str(m)} for m in rz['title']])

TypeError: sequence item 0: expected str instance, set found